#Оценка качества изображений

**Постановка задачи**: Фотографии загружаемые поставщиками WB имеют разное качество: На одних может быть сложный фон, на каких-то фотографиях часть объекта не попала в кадр и.т.п. Для последующей работы с такими данными, например при использовании алгоритмов поиска по фото надо знать типs дефектов/овособенностей которые присутствуют на изображении.

<img src ="https://ml.gan4x4.ru/wb/quality/content/samples.png" width="800">


Всего 6 типов особенностей:  

* untidy,
* angle-composition,
* background,
* crop,
* text,
* multiple-objects

и один класс для изображений без дефектов
* good-image.

При этом изображение может содержать несколько видов дефектов.


Задача:

Требуется создать модель которая будет определять список дефектов для для изображения.

# Данные

По [ссылке](https://ml.gan4x4.ru/wb/quality/5000/student_5000.zip) доступен архив содержащий 5000 изображений и разметку.

Оригинальные изображения имели размер 900x1200 в датасете их разрешение уменьшено вдвое. Кроме изображений в архиве находиться csv файл c разметкой.
В первой колонке имя файла с изображением (без расширения), в остальных колонках названия классов к которым относиться изображение:

```
  18715,text,multiple-objects,,
  5259,text,background,,
  8932,background,,,
  ...

```

# Порядок выполнения задания

Задание рекомендуется выполнять по шагам:

1. Познакомьтесь с данными
2. Выберите метрику для оценки результата
3. Проведите анализ состояния вопроса, изучите существующие модели которые можно использовать для решения задачи
4. Проведите EDA, опишите особенности данных и проблемы которые они могут за собой повлечь
5. Подготовьте данные для обучения
6. Выберите baseline модель, оцените качество её работы на данном датасете.
7. Попробуйте улучшить значение метрики используя другую модель. Возможно обучив/дообучив ее.
8. Оцените быстродействие выбранной модели
9. Дайте оценку полученному результату.


**Важно!**

Блокнот должен содержать весь необходимый код для запуска финальной модели. Если для запуска требуется подгрузка весов, все ссылки длжны работать не только в вашем аккаунте но и в аккаунте преподавателя.

## Анализ данных (EDA)

In [2]:
!wget https://ml.gan4x4.ru/wb/quality/5000/student_5000.zip
!unzip student_5000.zip

7student_5000.zip      40% [===========>                  ]   43.92M    6.64MB/s87student_5000.zip      40% [===========>                  ]   44.90M    6.59MB/s87student_5000.zip      41% [===========>                  ]   45.87M    6.62MB/s87student_5000.zip      42% [===========>                  ]   46.62M    6.50MB/s87student_5000.zip      43% [============>                 ]   47.60M    6.53MB/s87student_5000.zip      44% [============>                 ]   48.60M    6.59MB/s87student_5000.zip      45% [============>                 ]   49.31M    6.57MB/s87student_5000.zip      45% [============>                 ]   50.20M    6.62MB/s87student_5000.zip      46% [============>                 ]   50.96M    6.64MB/s87student_5000.zip      47% [=============>                ]   51.95M    6.70MB/s87student_5000.zip      48% [=============>                ]   52.90M    6.73MB/s87student_5000.zip      49% [=============>                ]   53.82M    6.74MB/s87st

In [1]:
# Dataset:
# 5000 /
#  ├── images /
#  │     ├── 0.jpg
#  │     ├── 1.jpg
#  │     ├── ...
#  ├── 5000.csv

# 5000.csv: 
# 18715,text,multiple-objects,,
# 5259,text,background,,
# 5259,text,background,,

import datasets
import pandas as pd

features = datasets.Features({
    'image': datasets.Image(),
    'text': datasets.Value('bool'),
    'multiple-objects': datasets.Value('bool'),
    'background': datasets.Value('bool'),
    'crop': datasets.Value('bool'),
    'angle-composition': datasets.Value('bool'),
    'untidy': datasets.Value('bool'),
    'good-image': datasets.Value('bool')
})

row_to_features = lambda row: {
    'image': f"5000/images/{row[0]}.jpg",
    'text': 'text' in row.values,
    'multiple-objects': 'multiple-objects' in row.values,
    'background': 'background' in row.values,
    'crop': 'crop' in row.values,
    'angle-composition': 'angle-composition' in row.values,
    'untidy': 'untidy' in row.values,
    'good-image': 'good-image' in row.values
}

df = pd.read_csv('5000/5000.csv', header=None)
df = df.fillna('')
as_list = df.apply(lambda x: row_to_features(x), axis=1).values.tolist()
dataset = datasets.Dataset.from_list(as_list, features=features)

In [ ]:
# find non-RGB

non_rgb = []
for i, row in enumerate(dataset):
    if row['image'].mode != 'RGB':
        print(row["image"].filename)
        non_rgb.append(i)

In [3]:
# convert to RGB
from PIL import Image

for i in non_rgb:
    img = Image.open(dataset[i]['image'].filename)
    img = img.convert('RGB')
    img.save(dataset[i]['image'].filename)

# Метрики

In [29]:
# Put your code here

## Baseline

In [4]:
from transformers import ViTForImageClassification, ViTImageProcessorFast

label_list = ['good-image', 'angle-composition', 'background', 'crop', 'text', 'multiple-objects', 'untidy']
id2label = {i: label for i, label in enumerate(label_list)}
label2id = {label: i for i, label in enumerate(label_list)}

processor = ViTImageProcessorFast.from_pretrained('facebook/dino-vitb16')
model = ViTForImageClassification.from_pretrained('facebook/dino-vitb16',
                                                  problem_type="multi_label_classification",
                                                  num_labels=len(label_list),
                                                  id2label=id2label,
                                                  label2id=label2id)

Some weights of ViTForImageClassification were not initialized from the model checkpoint at facebook/dino-vitb16 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Решение

In [5]:
import torch


def tokenize_function(examples):
    batch_size = len(examples['image'])
    labels_tensors = torch.zeros((batch_size, len(label_list)))

    for i in range(batch_size):
        for label in label_list:
            if examples[label][i]:
                labels_tensors[i][label2id[label]] = 1
    examples['pixel_values'] = processor(images=examples['image'], return_tensors="pt").pixel_values.squeeze()
    examples['labels'] = labels_tensors
    return examples


dataset_for_training = (dataset
                        .map(tokenize_function, batched=True, keep_in_memory=True)
                        .select_columns(['pixel_values', 'labels']))
dataset_for_training.set_format(type='torch', columns=['pixel_values', 'labels'])

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [6]:
import evaluate
import numpy as np

clf_metrics = evaluate.combine(["accuracy", "f1", "precision", "recall"])


def sigmoid(x):
    return 1 / (1 + np.exp(-x))


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = sigmoid(predictions)
    predictions = (predictions > 0.5).astype(int).reshape(-1)
    return clf_metrics.compute(predictions=predictions, references=labels.astype(int).reshape(-1))

In [7]:
splitted = dataset_for_training.train_test_split(test_size=0.2)

In [10]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir='./results',  # output directory
    num_train_epochs=10,  # total number of training epochs
    per_device_train_batch_size=256,  # batch size per device during training
    per_device_eval_batch_size=256,  # batch size for evaluation
    dataloader_num_workers=40,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    warmup_steps=500,  # number of warmup steps for learning rate scheduler
    weight_decay=0.01,  # strength of weight decay
    logging_dir='./logs',  # directory for storing logs
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=splitted["train"],
    eval_dataset=splitted["test"],
    compute_metrics=compute_metrics,
)

Detected kernel version 4.15.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [11]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,0.089072,0.970571,0.907040,0.944549,0.872396
2,No log,0.089183,0.970000,0.905063,0.944340,0.868924
3,No log,0.090704,0.970143,0.905643,0.943556,0.870660
4,No log,0.093510,0.970857,0.908108,0.943820,0.875000
5,No log,0.094727,0.971000,0.908517,0.944705,0.875000
6,No log,0.102382,0.970571,0.907374,0.941231,0.875868
7,No log,0.105515,0.970857,0.908602,0.938889,0.880208


KeyboardInterrupt: 

## Оценка результата

In [ ]:
# Put your code here

## Вывод

...

# Тестовый блок для проверки

Поместите сюда весь необходимый код для тестирования вашей модели на новых данных. Убедитесь что

- Импортируются все библиотеки и классы
- Подгружабтся веса с внешних ресурсов
- Происходит рассчет метрик
...

In [ ]:
# Put your code here